In [1]:
# Import libraries 
import numpy as np
import pandas as pd

In [21]:
# Define a minimum and maximum VDOT range
VDOT_Min = 40
VDOT_Max = 85 

# Define functions to calculate predicted race time based on VDOT value
def VO2(velocity):
    return -4.6 + 0.182258 * velocity + 0.000104 * velocity ** 2

def Velocity(VO2):
    return 29.54 + 5.000663 * VO2 - 0.007546 * VO2 ** 2

def VelocityPrime(VO2):
    return 5.000663 - 2 * (0.007546) * VO2

def FVDOT(time):
    return 0.8 + 0.1894393 * np.exp(-0.012778 * time) + 0.2989558 * np.exp(-0.1932605 * time)

def FVDOTPrime(time):
    return (-0.012778) * 0.1894393 * np.exp(-0.012778 * time) + (-0.1932605) * 0.2989558 * np.exp(-0.1932605 * time)

def VDOT(time, distance):
    velocity = distance / time
    return VO2(velocity) / FVDOT(time)

def Function(time, distance, VDOT):
    return time * Velocity(VDOT * FVDOT(time)) - distance

def FunctionPrime(time, VDOT):
    return Velocity(VDOT * FVDOT(time)) + time * VelocityPrime(VDOT * FVDOT(time)) * VDOT * FVDOTPrime(time)

def RacePrediction(VDOT, distance, time, error):
    while(abs(Function(time, distance, VDOT)) > error):
        time = time - (Function(time, distance, VDOT) / FunctionPrime(time, VDOT))
    return time

# Converts minutes to h:mm:ss format
def ConvertMinutes(minutes):
    seconds = minutes * 60
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    return '%d:%02d:%02d' % (h, m, s)

# Define function to produce VDOT Table based on VDOT range
def VDOT_Table():
    distance = [1500, 3000, 5000, 10000, 21098, 42195]
    time = [5, 10, 20, 40, 90, 180]
    race = []

    for i in range(VDOT_Min, VDOT_Max + 1):

        results = [i]

        index = 0

        for j in distance:

            prediction = RacePrediction(i, j, time[index], 0.0001)

            prediction = ConvertMinutes(prediction)

            results.append(prediction)

            index = index + 1

        race.append(results)
    
    return race


In [22]:
# Display VDOT Table
vdot_table = pd.DataFrame(VDOT_Table(), columns=['VDOT', '1500m', '3000m', '5000m', '10000m', 'Half', 'Marathon'])
vdot_table.set_index('VDOT')

,1500m,3000m,5000m,10000m,Half,Marathon
VDOT,,,,,,
40,0:06:35,0:14:02,0:24:07,0:50:03,1:50:59,3:49:45
41,0:06:26,0:13:44,0:23:37,0:49:00,1:48:40,3:45:08
42,0:06:18,0:13:27,0:23:08,0:48:00,1:46:27,3:40:43
43,0:06:10,0:13:11,0:22:41,0:47:03,1:44:19,3:36:27
44,0:06:03,0:12:55,0:22:14,0:46:08,1:42:17,3:32:22
45,0:05:55,0:12:40,0:21:49,0:45:15,1:40:19,3:28:26
46,0:05:48,0:12:25,0:21:25,0:44:25,1:38:26,3:24:39
47,0:05:42,0:12:11,0:21:01,0:43:36,1:36:38,3:21:00
48,0:05:35,0:11:58,0:20:39,0:42:49,1:34:53,3:17:29
